## 准备数据

In [12]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [13]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [14]:
class myModel:  
    def __init__(self):  
        ####################
        '''声明模型对应的参数'''
        #################### 
        self.W1 = tf.Variable(tf.random.normal([784, 128]))  
        self.b1 = tf.Variable(tf.zeros([128]))  
        self.W2 = tf.Variable(tf.random.normal([128, 64]))  
        self.b2 = tf.Variable(tf.zeros([64]))  
        self.W3 = tf.Variable(tf.random.normal([64, 10]))  
        self.b3 = tf.Variable(tf.zeros([10]))  
  
    def __call__(self, x):  
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])  
          
        # Forward pass  
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)  
        h2 = tf.nn.relu(tf.matmul(h1, self.W2) + self.b2)  
        logits = tf.matmul(h2, self.W3) + self.b3  
          
        return logits

model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [15]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [16]:
train_data, test_data = mnist_dataset()
for epoch in range(80):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 561.8199 ; accuracy 0.10158333
epoch 1 : loss 414.17654 ; accuracy 0.116
epoch 2 : loss 348.80893 ; accuracy 0.14235
epoch 3 : loss 302.6072 ; accuracy 0.16725
epoch 4 : loss 265.53107 ; accuracy 0.19308333
epoch 5 : loss 235.23187 ; accuracy 0.21828334
epoch 6 : loss 210.67897 ; accuracy 0.24285
epoch 7 : loss 190.9665 ; accuracy 0.26843333
epoch 8 : loss 175.08167 ; accuracy 0.29186666
epoch 9 : loss 162.07068 ; accuracy 0.31295
epoch 10 : loss 151.29282 ; accuracy 0.33306667
epoch 11 : loss 142.27943 ; accuracy 0.35006666
epoch 12 : loss 134.55345 ; accuracy 0.36691666
epoch 13 : loss 127.85699 ; accuracy 0.38213333
epoch 14 : loss 121.955376 ; accuracy 0.39558333
epoch 15 : loss 116.71594 ; accuracy 0.40725
epoch 16 : loss 112.02459 ; accuracy 0.4182
epoch 17 : loss 107.81037 ; accuracy 0.42843333
epoch 18 : loss 103.984535 ; accuracy 0.43835
epoch 19 : loss 100.48204 ; accuracy 0.44741666
epoch 20 : loss 97.27261 ; accuracy 0.45595
epoch 21 : loss 94.314156 ; accura